In [ ]:
## 將 Mycard 平台所輸出的檔案轉成分析用的檔案

## 使用的資料表
# 1. df1 - suc_records: 每筆交易紀錄
# 2. df2 - user: 以 user id 為基準, 看 RFM 相關紀錄
# 3. df3 - Daily_per: 以日期為基準, 看每日活躍 和巴哈熱度

##其他
# 1. df_ - 原始 mycard 交易紀錄


# 使用套件
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import glob
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
#### DF1: 
# 1.讀 Mycard 交易紀錄資料

# 透過給定地址和模糊比對,把全部符合規則的交易紀錄檔案讀入至 allFiles
path =r"C:\Users\Ting\Desktop\mycard_paypal金流營收\MyCard_record" # use your path
allFiles = glob.glob(path + "/*.xlsx")

In [ ]:
# 先創造一個 dic 來紀錄要存的資料欄位 來改變樹入欄位的名稱
columns_name = {"交易日期 Transaction Date of Success":"Date_of_Success", 
                "交易成功時間 Success Time of Transaction":"Date_of_Success",
                "交易成功時間 Transaction Date of Success":"Date_of_Success",
                "金額 Unit Price":"Price","定價 List Price":"Price",
                "取消狀態 Canceled or not":"Canceled_or_not",
                "取消與否 Cancel or NO":"Canceled_or_not",
                "取消與否 Cancel or No":"Canceled_or_not",
                "使用者ID User ID":"User_ID",
                "會員/遊戲帳號 User Account/Game Account":"User_ID",
                "通路別 Distributor Type":"Channel","付費方式 Payment":"Channel"}

# 先透過表格名稱來判斷該檔案屬於哪個儲值管道
temp = pd.DataFrame()
temp2 = pd.DataFrame()
list_ = []

for file_ in allFiles:
    # 用檔案名稱來判斷屬於哪個儲值管道
    if "MyCard_ATM" in file_:
        # 重新命名欄位名稱
        temp = pd.read_excel(file_ ,skiprow=0).rename(columns=columns_name)
        # 只要特定幾個欄位
        temp2 = temp[["Date_of_Success","Price","Canceled_or_not","Channel","User_ID"]]
        # 把該 df 新加到 list 中
        list_.append(temp2)
    
    if "MyCard_Retailer" in file_:
        temp = pd.read_excel(file_,skiprow=0).rename(columns=columns_name)
        temp2 = temp[["Date_of_Success","Price","Canceled_or_not","Channel","User_ID"]]
        list_.append(temp2)
    
    if "MyCard_member" in file_:
        temp = pd.read_excel(file_,skiprow=0)
        temp['User_ID'] = 0
        temp["Channel"] = "Mycard_Member"
        temp = temp.rename(columns=columns_name)
        temp2 = temp[["Date_of_Success","Price","Canceled_or_not","Channel","User_ID"]]
        list_.append(temp2)
        
# 把存多個df的 list 資料轉 dataframe
df_ = pd.concat(list_)

In [ ]:
# 調整原始df 的資料型態
# 把原本的 df_ 保留住
records = df_.copy()

# 將 User_ID 轉成 文字
records["User_ID"] = records["User_ID"].apply(str)

# 調整 Canceled_or_not 的欄位值
# 確認是前狀況
print(records["Canceled_or_not"].value_counts())

def canceled(str_):
    if str_ == "未取消" or str_ =="否" or str_ =="NO":
        return "No"
    else:
        return "Yes"
    
records["Canceled_or_not"] = records["Canceled_or_not"].apply(canceled)

In [ ]:
## 調整欄位的時間格式
# 新增交易日 Date, 透過切資料找第一個值
records["Date"] = pd.to_datetime(records["Date_of_Success"].str.split(" ").str[0])

# 看資料共包含幾天
print(len(records["Date"].unique()))

## 確認時間格式的種類 (用空格來分辨
list3_ = []
for x in records["Date_of_Success"].tolist():
    list3_.append(len(x.split(" ")))

# set 找不重複的資料, 發現有兩種形式的資料
print("時間型態格式")
print(set(list3_))

# 把文字的交易時間轉成 時間格式
def time_type(x):
    if len(x.split(" ")) == 3:
        x = x.replace(" 上午 ",":AM:").replace(" 下午 ",":PM:")
        return dt.datetime.strptime(x,"%Y/%m/%d:%p:%I:%M:%S")
    if len(x.split(" ")) == 2:
        return dt.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")

records["Date_Time"] = records["Date_of_Success"].apply(lambda x: time_type(x))
print("看資料的時間週期")
records["Date_Time"].describe()

# 新增加 小時 & weekday & month 資料
def tohour_(x):
    return x.strftime("%H")

def toweekday_(x):
    return x.strftime("%a")

def tomonth_(x):
    return x.strftime("%m")

records["hour"] = records["Date_Time"].apply(tohour_)
records["weekday"] = records["Date_Time"].apply(toweekday_)
records["month"] = records["Date_Time"].apply(tomonth_)

In [ ]:
## 新加欄位 - 買虛寶所對應到的商品

# 透過每次儲值金額表示玩家購買之產品
#大品項
Package_list = [149,299,499,999,1249,2499]
PackageUpgrade_list = [350, 700, 750, 1100, 1500, 2200]
Pearl_list = [150, 300, 500, 1000, 2000, 3000, 5000, 10000]

# 購買細項 (產包升級或是購買產包
Starter_P_list = [149, 299]
Essential_P_list = [499, 999]
Luxurious_P_list = [1249, 2499]
StoE_P_list = [350,700]
StoL_P_list = [1100, 2200]
EtoL_P_list = [750, 1500]


# 將值替換成購買品項
def Purchase_item(price):
    if price in Package_list:
        return "Purchase_Package"
    if price in PackageUpgrade_list:
        return "Upgrade_Package"
    if price in Pearl_list:
        return "Purchase_Pearl"
    
def Purchase_detail(price):
    if price in Starter_P_list:
        return "Starter_P"
    if price in Essential_P_list:
        return "Essential_P"
    if price in Luxurious_P_list:
        return "Luxurious_P"
    if price in StoE_P_list:
        return "StoE_P"
    if price in StoL_P_list:
        return "StoL_P"
    if price in EtoL_P_list:
        return "EtoL_P"
    if price in Pearl_list:
        return "Purchase_Pearl"
    
records["Purchase_item"] = records["Price"].apply(Purchase_item)
print(records["Purchase_item"].value_counts())

records["Purchase_detail"] = records["Price"].apply(Purchase_detail)

In [ ]:
## 把儲值通路分類
ATM_list = ["銀行轉帳", "中華郵政WebATM",  "玉山銀行WebATM", "國泰世華MyATM",
            "中國信託WebATM", "台灣銀行WebATM",  "合作金庫WebATM",
            "新光銀行WebATM", "土地銀行WebATM","華南銀行WebATM", "第一銀行WebATM", 
            "兆豐銀行WebATM", "台新銀行WebATM",  "彰化銀行WebATM", "台北富邦WebATM", 
            "上海銀行WebATM","日盛銀行WebATM"]

Convience_Stores_list = ["全家以外之超商通路", "全家超商通路"]
Credit_Card_list = ["信用卡-台灣地區(3D驗證)"]
Micropayment_list = ["台灣大哥大電信", "中華電信839", "中華電信HiNet","遠傳電信帳單付費",
                     "中華電信市內電話輕鬆付","亞太電信","台灣之星","Seednet"]

Mycard_Member_list = ["Mycard_Member"]
General_Retailer_list = ["一般通路"]

def Payment_Type(pay):
    if pay in ATM_list:
        return "ATM"
    if pay in Convience_Stores_list:
        return "Convience_Stores"
    if pay in Credit_Card_list:
        return "Credit_Card"
    if pay in Micropayment_list:
        return "Micropayment"
    if pay in Mycard_Member_list:
        return "Mycard_Member"
    if pay in General_Retailer_list:
        return "General_Retailer"

records["Payment_type"] = records["Channel"].apply(Payment_Type)

##　把取消的交易拿掉
suc_records = records[records["Canceled_or_not"] =="No"]

In [ ]:
##### 創造 DF2
## 以 USER id 為依據的表單
## RFM 分析

# 已將 user 轉成 string 所以0 要用 ""
# 因為無法用 id 判斷 Mycard 會員，所以要將資料拿掉
user_temp = suc_records[suc_records["User_ID"] != "0"]

# 累計付款總額
Total_Amount = user_temp.groupby("User_ID").Price.sum()

# 累計付款次數
Total_Times = user_temp.groupby("User_ID").Price.count()

# 找出玩家上一次付款的時間
Previous_Date = user_temp.groupby("User_ID")["Date"].max()

# 找出玩家第一次付款的時間, 用第一次買的紀錄來判斷 (可能是產包或是珍珠)
First_Date = user_temp.groupby("User_ID")["Date"].min()

# 找出玩家在不同支付管道的購買次數 ATM , Convience_Stores , Credit_Card, Micropayment, General_Retailer
temp = user_temp.pivot_table(values="Price" , index="User_ID", columns="Payment_type", aggfunc="count")

#　再將付款紀錄和　user id 結合成新的 df
list_of_series = [Total_Amount, Total_Times, Previous_Date, First_Date]
temp_ = pd.concat(list_of_series,axis=1)
user = pd.concat([temp,temp_],axis=1).fillna(0)

# 重新命名欄位名稱
cols = [ "ATM_count", "CS_count", "Credit_count", "Retailer_count", "Micro_count",
        "Total_amount",'Total_count', "Previous_date" , "First_date"]
user.columns =cols

# 新增欄位, 已多久沒付款
user["Stopping_days"] = (dt.datetime(2018,3,31) - user["Previous_date"])/dt.timedelta(days=1)

# 新增欄位 已加入遊戲多久
user["Playing_days"] = (dt.datetime(2018,3,31) - user["First_date"])/dt.timedelta(days=1)

# 新增欄位 加入遊戲至上次附款日期時間
user["Paying_days"] = (user["Previous_date"] - user["First_date"])/dt.timedelta(days=1)

# 確認玩家購買產包型態
Pack_df = user_temp[user_temp["Purchase_item"] =="Purchase_Package"].copy()
Pack_df.columns = ["Pack_type" if x=="Purchase_detail" else x for x in Pack_df.columns]


# 讓玩家按總付款金額排序
user = user.sort_values(by="Total_amount", ascending= False).reset_index()

# 將 index 欄位命名成 User_ID
user.columns = ["User_ID" if x=="index" else x for x in user.columns]

user = pd.merge(user,Pack_df[["User_ID","Pack_type"]],on="User_ID",how="left").fillna("NA")

# 新增玩家傾向附款管道 (>0.6 算篇好, 用次數算 而不是總消費金額
def Pay_type(df):
    if (df["ATM_count"]/df["Total_count"]) >0.6:
        return "ATM"
    elif (df["CS_count"]/df["Total_count"]) >0.6:
        return "CS"
    elif (df["Credit_count"]/df["Total_count"]) >0.6:
        return "Credit_card"
    elif (df["Retailer_count"]/df["Total_count"]) >0.6:
        return "retailer"
    elif (df["Micro_count"]/df["Total_count"]) >0.6:
        return "Micro"
    else:
        return "multi_payments"
        
user["Prefer_payment"] = user.apply(Pay_type,axis=1)

# 新增購買珍珠次數
def pearl_count(df):
    if df["Pack_type"] == "NA":
        return df["Total_count"]
    else:
        return df["Total_count"] - 1
    
user["Pearl_count"] = user.apply(pearl_count, axis=1)

# 新增購買玩家屬性 (有無購買珍珠)
def paid_user_(x):
    if x>0:
        return "Paid_user"
    else:
        return "NonPaid_user"
    
user["User_type"] = user["Pearl_count"].apply(paid_user_)

# 把有付錢買過一次珍珠的用戶拉出
paid_user = user[user["User_type"]== "Paid_user"].copy()

In [ ]:
##### 創造 DF3
## 以每日為基準看遊戲和巴哈資料

# 匯入使用者人數 active user
path =r"C:\Users\Ting\Desktop\input_source\active_user.csv"
active_user = pd.read_csv(path,sep=",")
active_user.Date = pd.to_datetime(active_user.Date)

# 把每日的營收整理
daily_revenue = suc_records.groupby("Date").Price.sum().reset_index()
daily_paid_user = suc_records.groupby(pd.Grouper(freq="d",key="Date")).User_ID.count().reset_index()
daily_paid_user.columns = ["Date", "Paid_user"]

# 合併兩個資料
temp = pd.merge(active_user, daily_revenue,how="left",on="Date")
Daily_per = pd.merge(temp, daily_paid_user, how="left", on= "Date")

Daily_per["NRU%"] = Daily_per["NRU_PKG"] / Daily_per["DAU_PKG"]
Daily_per["Paid%"] = Daily_per["Paid_user"] / Daily_per["DAU_PKG"]
Daily_per



STEP2: 資料視覺化
PART1: 整體營運狀況圖表

In [ ]:
# 確認不同管道的營收狀況
src = suc_records.groupby("Payment_type").Price.sum().reset_index()
src["Revenue%"] = src["Price"]/src["Price"].sum()
src["Revenue"] = src["Price"]/1000000
print(src)

plt.style.use("ggplot")
plt.figure(figsize=(10,5))
plt.title('Revenue by Channel')
sns.barplot(x="Payment_type", y="Revenue", data = src).set(xlabel="Payment Channel" , ylabel="M, NTD")

In [ ]:
# 買珍珠的面額概況
temp = suc_records[suc_records["Purchase_item"] =="Purchase_Pearl"]
src = temp.groupby("Price").User_ID.count().fillna(0).reset_index()
print(src)

plt.style.use("ggplot")
plt.figure(figsize=(10,5))
plt.title('Revenue by Denomination')
sns.barplot(x="Price", y="User_ID", data = src).set(xlabel="Denomination" , ylabel="Times")


In [ ]:
# 確認產包購買狀況分布, 
# 要先指定三個產包條件 (isin)
src = suc_records.groupby("Purchase_detail").Price.count().reset_index()
src2 = src[src.Purchase_detail.isin(["Essential_P", "Luxurious_P","Starter_P"])]
print(src2)
plt.figure(figsize=(10,5))

plt.title('PKG Count')
sns.barplot(x="Purchase_detail", y="Price", data = src2).set(xlabel="Package Purchased" , ylabel="Count")

In [ ]:
# 畫每日玩家和營收狀態
plt.figure(figsize=(10,5))
plt.style.use("ggplot")
plt.title("Daily Active User and Revenue")

a = plt.plot(Daily_per["Date"], Daily_per["DAU_PKG"], label="Daily User", color="teal")
plt.ylabel("User")
plt.legend(loc=9)


plt.twinx()
b = plt.bar(Daily_per["Date"], Daily_per["Price"]/1000, label="Daily Revenue (K, NTD)", 
             color="lightslategray", alpha =0.6)


plt.ylabel("K, NTD")
plt.xlabel("Date")
plt.legend(loc=1)

In [ ]:
# 畫DAU 和每日付費玩家數目
plt.figure(figsize=(10,5))
plt.style.use("ggplot")
plt.title("Daily Active User and Bahamut Impression")

a = plt.plot(Daily_per["Date"], Daily_per["DAU_PKG"], label="Daily User", color="teal")
b = plt.bar(Daily_per["Date"], Daily_per["Baha_imp"]/25, label="Bahamut impression", 
             color="lightslategray", alpha =0.6)

plt.ylabel("DAU")
plt.xlabel("Date")
plt.legend()

In [ ]:
# 畫DAU 和每日新增加玩家 (DAU & NRU
plt.figure(figsize=(10,5))
plt.style.use("ggplot")
plt.title("Daily Active User and NRU%")

a = plt.plot(Daily_per["Date"], Daily_per["DAU_PKG"], label="Daily User", color="teal")
plt.ylabel("DAU")
plt.legend(loc=9)


plt.twinx()
b = plt.bar(Daily_per["Date"], Daily_per["NRU%"], label="NRU/DAU", 
             color="lightslategray", alpha =0.6)

plt.ylim(0,0.025)
plt.ylabel("NRU%")
plt.xlabel("Date")
plt.legend(loc=1)


In [ ]:
# 付費玩家的人數變化
src = suc_records.groupby(pd.Grouper(freq='m',key="Date_Time")).agg({"User_ID":"count","Price":"sum"}).reset_index()
src["Monthly_ARPPU"] = round(src["Price"]/src["User_ID"],2)
src.columns = ["Month", "Monthly_user", "Monthly_revenue", "Monthly_ARPPU"]
print(src)


plt.style.use("ggplot")
plt.figure(figsize=(10,5))
plt.plot(src["Month"],src["Monthly_revenue"]/1000000, label = "Revenue (M,NTD)", color= "teal")
plt.plot(src["Month"],src["Monthly_user"]/1000, label = "Paid User (K)", color= "lightslategray")
plt.xlabel("Date")
plt.legend()
plt.title("Monthly Revenue from MyCard")

In [ ]:
# 畫DAU 和巴哈人氣
plt.figure(figsize=(10,5))
plt.style.use("ggplot")
plt.title("Daily Active User and Paid%")

a = plt.plot(Daily_per["Date"], Daily_per["DAU_PKG"], label="Daily User", color="teal")
plt.ylabel("DAU")
plt.legend(loc=9)

plt.twinx()
b = plt.bar(Daily_per["Date"], Daily_per["Paid%"], label="Paid User/DAU", 
             color="lightslategray", alpha =0.6)

plt.ylim(0,0.2)
plt.ylabel("Paid%")
plt.xlabel("Date")
plt.legend(loc=1)

In [ ]:
# 看每月營收來源比例 (珍珠 / 產包升級 / 產寶)
src = suc_records.pivot_table(index=pd.Grouper(freq="m",key="Date"),columns = "Purchase_item", values="Price", aggfunc="sum").reset_index()

N = len(src)
Package_ = src["Purchase_Package"]/1000000
Pearl_ = src["Purchase_Pearl"]/1000000
Upgrad_ = src["Upgrade_Package"]/1000000

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence


plt.style.use("ggplot")
plt.figure(figsize=(10,5))
p1 = plt.bar(ind, Pearl_, width, label="Pearl")
p2 = plt.bar(ind, Package_, width,bottom=Pearl_, label="Package")
p3 = plt.bar(ind, Upgrad_, width,bottom=Pearl_+Package_, label="Upgrade")

def month_(x):
    return x.strftime("%y-%m")

plt.ylabel('M, NTD')
plt.title('Monthly Revenue by item purchased')
plt.xticks(ind,src["Date"].apply(month_),rotation=20)
plt.legend()


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % float(height),
                ha='center', va='bottom')
        
autolabel(p2)

print(round((Package_ / (Package_+Pearl_+Upgrad_ )),4))

In [ ]:
# 看每月營收的儲值管道分布
src = suc_records.pivot_table(index=pd.Grouper(freq="m",key="Date"),columns = "Payment_type", values="Price", aggfunc="sum").reset_index()

N = len(src)
MY_ = src["Mycard_Member"]/1000000
Cd_ = src["Credit_Card"]/1000000
cs_ = src["Convience_Stores"]/1000000
re_ = src["General_Retailer"]/1000000
Mipy_ = src["Micropayment"]/1000000
atm_ = src["ATM"]/1000000

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

print(MY_/(MY_+Cd_+cs_+re_+Mipy_+atm_))
print(Cd_/(MY_+Cd_+cs_+re_+Mipy_+atm_))
print((MY_ + Cd_)/(MY_+Cd_+cs_+re_+Mipy_+atm_))

plt.style.use("ggplot")
plt.figure(figsize=(10,5))
p1 = plt.bar(ind, MY_, width, label= "Mycard Member")
p2 = plt.bar(ind, Cd_, width, bottom= MY_, label= "Credit Card")
p3 = plt.bar(ind, cs_, width, bottom= MY_+Cd_, label= "Convience Stores")
p4 = plt.bar(ind, re_, width, bottom= MY_+Cd_+cs_, label= "Retailer")
p5 = plt.bar(ind, Mipy_, width, bottom = MY_+Cd_+cs_+re_, label= "Micropayment")
p6 = plt.bar(ind, atm_, width, bottom = MY_+Cd_+cs_+re_+Mipy_, label= "ATM")

plt.ylabel('M, NTD')
plt.title('Monthly Revenue by Channel')
plt.xticks(ind,src["Date"].apply(month_),rotation=20)
plt.legend()

In [ ]:
# 看每月營收分別來自哪個儲值管道
src["Revenue"] = src["ATM"] + src["Convience_Stores"] + src["Credit_Card"] + src["General_Retailer"] + src["Micropayment"] + src["Mycard_Member"]
src["ATM%"] = src["ATM"]/src["Revenue"]
src["Convience_Stores%"] = src["Convience_Stores"]/src["Revenue"]
src["Credit_Card%"] = src["Credit_Card"]/src["Revenue"]
src["General_Retailer%"] = src["General_Retailer"]/src["Revenue"]
src["Micropayment%"] = src["Micropayment"]/src["Revenue"]
src["Mycard_Member%"] = src["Mycard_Member"]/src["Revenue"]

N = len(src)
MY_ = src["Mycard_Member%"]
Cd_ = src["Credit_Card%"]
cs_ = src["Convience_Stores%"]
re_ = src["General_Retailer%"]
Mipy_ = src["Micropayment%"]
atm_ = src["ATM%"]

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

print(src)

plt.style.use("ggplot")
plt.figure(figsize=(10,5))
p1 = plt.bar(ind, MY_, width, label= "Mycard Member")
p2 = plt.bar(ind, Cd_, width, bottom= MY_, label= "Credit Card")
p3 = plt.bar(ind, cs_, width, bottom= MY_+Cd_, label= "Convience Stores")
p4 = plt.bar(ind, re_, width, bottom= MY_+Cd_+cs_, label= "Retailer")
p5 = plt.bar(ind, Mipy_, width, bottom = MY_+Cd_+cs_+re_, label= "Micropayment")
p6 = plt.bar(ind, atm_, width, bottom = MY_+Cd_+cs_+re_+Mipy_, label= "ATM")

plt.title('Monthly Revenue % by Channel')
plt.xticks(ind,src["Date"].apply(month_),rotation=20)

PART3: Customer behavior

In [ ]:
# 確認前10 20 30 40 50 ... 100 的玩家貢獻營收的比例
# step1. 平分一個範圍  
num_list = list(np.linspace(0,1,11))
num_list.pop(0)

# step2. 透過 np.array_split 把一個 df 切成十等分
cum_list = []
for i in range(10):
    cum_list.append(np.array_split(user,10)[i].Total_amount.sum())

# step3. 計算出玩家數量比例和消費金額比例
src = pd.DataFrame({"Amount": cum_list,"User%": num_list})
src["Amount%"] = src["Amount"]/src["Amount"].sum()
src["Cum_Amount%"] = src["Amount%"].cumsum()

print(src)

plt.style.use("ggplot")
plt.figure(figsize=(10,5))
plt.plot(src["User%"], src["Cum_Amount%"])
plt.xlabel("User %")
plt.ylabel("Revenue %")

# 標註線上的值, 並將數值轉百分比
for a,b in zip(src["User%"],src["Cum_Amount%"]): 
    plt.text(a, b, "{:.1%}".format(round(b,3)))

plt.title("How Whale spend in game")

In [ ]:
# 確認不同小時的營收狀況
src = suc_records.groupby("hour").Price.sum().reset_index()
src["Revenue%"] = src["Price"]/src["Price"].sum()
print(src)

plt.figure(figsize=(10,5))
sns.barplot(x="hour", y="Revenue%", data = src)
plt.ylabel('')
plt.title('When gamer purchase item')

In [ ]:
# 確認不同天的營收狀況
# 重新排序星期順序
src = suc_records.groupby("weekday").Price.sum().reset_index()
src["Revenue%"] = src["Price"]/src["Price"].sum()

day_ = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
src['weekday'] = pd.Categorical(src['weekday'],day_)
src= src.sort_values("weekday")

print(src)

plt.figure(figsize=(10,5))
sns.barplot(x="weekday", y="Revenue%", data = src)
plt.ylabel('')
plt.title('When gamer purchase item')

In [ ]:
# 看星期和小時的付費次數狀況
src = suc_records.pivot_table(columns="hour", index="weekday", values="Price", aggfunc="count")

f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(src, annot=False, linewidths=.5, ax=ax ,center=7000, cmap="YlGnBu")

plt.title('When gamer spend money (Times)')

In [ ]:
# 看星期和小時的付費金額狀況
src = suc_records.pivot_table(columns="hour", index="weekday", values="Price", aggfunc="sum")

f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(src, annot=False, linewidths=.5, ax=ax , cmap="YlGnBu")

plt.title('When gamer spend money (Amount)')

In [ ]:
# 看不同產包購買者 習慣的支付管道為河
src = user.pivot_table(columns="Prefer_payment", index = "Pack_type", values="User_ID", aggfunc="count")
src["TotalUser"] = src["ATM"] + src["CS"] + src["Credit_card"] + src["Micro"] + src["multi_payments"] + src["retailer"]

for i in src.columns[:6]:
    str_ = i + "%"
    src[str_] = src[i]/src["TotalUser"]

# 把 NA 資料拿掉
src = src.drop(["NA"])
print(src)

N = len(src)
Cd_ = src["Credit_card%"]
cs_ = src["CS%"]
re_ = src["retailer%"]
Mipy_ = src["Micro%"]
atm_ = src["ATM%"]
multi_ = src["multi_payments%"]

ind = np.arange(len(src))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

plt.style.use("ggplot")
plt.figure(figsize=(10,5))

p1 = plt.bar(ind, Cd_, width, label= "Credit Card",color="salmon")
p2 = plt.bar(ind, Mipy_, width, bottom= Cd_, label= "Micropayment", color="navajowhite")
p3 = plt.bar(ind, cs_, width, bottom= Cd_+Mipy_, label= "Convience Stores", color="lightblue")
p4 = plt.bar(ind, multi_, width, bottom = Cd_+Mipy_+cs_, label= "Multi Payment", color="lightgray")
p5 = plt.bar(ind, re_, width, bottom = Cd_+Mipy_+cs_+multi_, label= "Retailer", color="steelblue")
p6 = plt.bar(ind, atm_, width, bottom = Cd_+Mipy_+cs_+multi_+re_, label= "ATM", color="gold")

label_ = ["Essential Package","Luxurious Package","Starter Package"]
plt.title('Monthly Revenue % by Channel')
plt.xticks(ind, label_, rotation=20)
plt.legend()